# Assigment 4

This assigment will be graded if everything works well. I will run the script as once and everything should be done without errors and mistakes. I should be able to run your scripts in my computer and get all the results. **USE RELATIVE PATHS**. An error or exception or anything that breaks the code will means NO GRADE (0). Additionally, you are not able to modify any file handly. It also means NO GRADE (0). Comment everything you think will help others read your script. We expect 0 errors using GitHub. Everything will be graded!

**ASK EVERYTHING! WE ARE HERE TO HELP YOU!**

In this path **..\_data\sbs\B_RawData\bancos** you will find scraped data from [this link](https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=B). We get all the information of the last available day of every each month.

1. Import all the data and generate a column named as `date_info` that should have the day to information corresponds.
2. Append all this datasets and generate a unique dataframe. This newdataset should have information at `rate interest` and `date` level. The columns should be the name of the banks. Be careful since not all the excel files have the same format. **It is totally prohibited to manipulate manually the excel files. This kind of action means NO GRADE on this project.**
3. What are the top 5 banks each year with the highest interest rate at `Préstamos hipotecarios para vivienda`, `Consumo -
Tarjetas de Crédito`. Present a dataframe with these variables: `year`, `rate_concept`, `banks`, `rate_value`.
4. We want to save this dataset in the folder **_output/sbs/group_#**, but we want to save a file per bank. We want to have the information disaggregated at the bank level. Please, save your files with the name of the bank. Avoid blank spaces and use only lowercase letters. Generate the folder of your group using python code. **Hint: os library**

### SOLUTIONS

#### Part 1.

In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
# Set relative and full paths

path = '../../_data/sbs/B_RawData/bancos/'
fullpath = os.path.abspath('../../_data/sbs/B_RawData/bancos/')

In [46]:
# Import excel files and add a column called "date_info", which takes information from the excel files names.

dfs = []
for file in os.listdir(path):
    date_info = file[file.rindex('table_clean_')+12:]
    date_info = date_info[:-5]
    table_names_path = os.path.join(fullpath, f'{file}')
    data = pd.read_excel(table_names_path)
    data['date_info'] = date_info
    dfs.append(data)

#### Part 2. 

In [47]:
# Loop to homogenize the excel files

for i, df in enumerate(dfs):
    if dfs[i].shape[0] < 25:
        dfs[i] = dfs[i].T
        new_colnames = dfs[i].iloc[0]
        dfs[i] = dfs[i][1:]
        dfs[i].columns = new_colnames
        dfs[i] = dfs[i].reset_index()
        dfs[i].columns.values[0] = "first_col"
        dfs[i].columns.values[1] = "second_col"
        dfs[i].columns.values[2] = "third_col"
        dfs[i].columns.values[-1] = "last_col"
        dfs[i][dfs[i]['last_col'].isnull()]=dfs[i][dfs[i]['last_col'].isnull()].shift(axis=1)
        dfs[i] = dfs[i].shift(-1, axis=1).dropna(axis=1)
        dfs[i]['catamaran'] = dfs[i].iloc[:,-1:]
        if dfs[i].shape[1] > 1: 
            dfs[i]['date_info'] = dfs[i]['catamaran'].iloc[len(dfs[i].index) - 1]
            dfs[i]['Tasa Anual (%)'] = dfs[i]['first_col'] + dfs[i]['third_col']
            dfs[i] = dfs[i].drop(['first_col', 'second_col', 'third_col', 'catamaran'], axis=1)
            dfs[i] = dfs[i].iloc[:-1 , :]
            dfs[i].index.names = ['index']

In [48]:
# Append vertically the excel files

data = pd.concat(dfs)

In [49]:
# Order columns

data = data.reindex(columns =['Tasa Anual (%)', 'date_info', 'TRABAJO',
                              'BBVA', 'BANCO DE COMERCIO', 'BANCO DE CREDITO',
                              'BANCO PICHINCHA', 'BANBIF', 'STANDARD CHARTERED',
                              'BANCO SUDAMERICANO', 'WIESE SUDAMERIS', 'BANK BOSTON',
                              'BNP PARIBAS EL', 'CITIBANK DEL PERU','INTERBANK', 
                              'MIBANCO','SCOTIABANK PERU', 'HSBC', 'BANCO FALABELLA', 
                              'SANTANDER PERU','BANCO RIPLEY', 'ALFIN BANCO', 
                              'DEUTSCHE BANK PERU','Promedio del Sistema', 'Comercio', 
                              'Crédito', 'Pichincha', 'BIF','Scotiabank', 'Citibank', 
                              'Interbank', 'Mibanco', 'GNB', 'Falabella', 'Santander', 
                              'Ripley', 'Azteca', 'ICBC', 'Bank of China', 'Promedio',
                              'Continental', 'Financiero', 'Banco GNB', 'Deutsche', 
                              'Cencosud', 'CAT', 'Alfin', 'B SANTANDER CENTRAL', 'Azteca  *',
                              'HSBC(*)', 'Financiero  *', 'BCI'])

In [51]:
data.to_excel('final_data.xlsx')

## Part 3.

In [52]:
df = pd.read_excel(r'final_data.xlsx')
df.head()

,Unnamed: 0,Tasa Anual (%),date_info,TRABAJO,BBVA,BANCO DE COMERCIO,BANCO DE CREDITO,BANCO PICHINCHA,BANBIF,STANDARD CHARTERED,...,Banco GNB,Deutsche,Cencosud,CAT,Alfin,B SANTANDER CENTRAL,Azteca *,HSBC(*),Financiero *,BCI
0,0,Avances en Cta.Corriente-,26_2_2004,38.92,-,73.34,-,39.08,-,31.73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Sobregiros-,26_2_2004,48.79,138.18,89.00,-,68.30,25.00,58.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Dsctos. y préstamos hasta 30 días-,26_2_2004,8.19,47.46,4.62,15.87,14.43,-,30.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Dsctos. y préstamos 31 - 90 días-,26_2_2004,5.94,21.61,7.19,15.71,10.91,-,11.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Dsctos. y préstamos 91 - 180 días-,26_2_2004,10.00,19.71,5.34,15.82,10.71,-,9.98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#DEBEMOS PERCATARNOS que hay datos de múltiples años, será necesario ubicar el dato correspondiente a cada ano y luego generr el ranking para las tres variables requerida

In [8]:
index = df.index
print(index)

RangeIndex(start=0, stop=9154, step=1)


In [9]:
df.iloc[:,0:2]

,Tasa Anual (%),date_info
0,Avances en Cta.Corriente-,26_2_2004
1,Sobregiros-,26_2_2004
2,Dsctos. y préstamos hasta 30 días-,26_2_2004
3,Dsctos. y préstamos 31 - 90 días-,26_2_2004
4,Dsctos. y préstamos 91 - 180 días-,26_2_2004
...,...,...
9149,Préstamos no Revolventes para libre disponibil...,31_8_2022
9150,Préstamos no Revolventes para libre disponibil...,31_8_2022
9151,Créditos pignoraticios,31_8_2022
9152,Hipotecarios,31_8_2022


In [10]:
type(df.iloc[9153])

pandas.core.series.Series

In [11]:
serie_d = df.iloc[9153]
df2 = serie_d.to_frame() #convertimos la serie a dataframe
df2.drop(['Tasa Anual (%)','date_info','TRABAJO'],axis=0,inplace=True) #eliminamos las primeras filas que no contienen información de los bancos
df2.rename(columns = {9153:'Tasa anual: Préstamos hipotecarios para vivienda'}, inplace = True) #cambiamos el nombre de la fila que tiene la información de tasas de interés
df2= df2.reset_index() #generamos un nuevo índice 
df2.rename(columns = {'index':'Banco'}, inplace = True) #y cambiamos la denominación de la nueva fila con nombres de los bancos
df2['Tasa anual: Préstamos hipotecarios para vivienda'] = df2['Tasa anual: Préstamos hipotecarios para vivienda'].replace(['-','s.i.'], 0) #reemplazamos la información que es texto con valores nulos
df2['Tasa anual: Préstamos hipotecarios para vivienda'] = df2['Tasa anual: Préstamos hipotecarios para vivienda'].fillna(0) # y convertimos los missings en valores nulos
df2['Tasa anual: Préstamos hipotecarios para vivienda'] = df2['Tasa anual: Préstamos hipotecarios para vivienda'].astype(float) #convertimos la columna de tasas a formato float
df2 = df2.sort_values(by="Tasa anual: Préstamos hipotecarios para vivienda",ascending=False) #ordenamos el dataframe ascendentemente
df2

,Banco,Tasa anual: Préstamos hipotecarios para vivienda
28,Mibanco,15.54
0,BBVA,9.28
22,Crédito,9.27
24,BIF,9.23
36,Promedio,9.13
21,Comercio,9.08
27,Interbank,8.73
23,Pichincha,8.72
25,Scotiabank,8.61
29,GNB,8.03


In [12]:
top5_hipo= df2.sort_values(by="Tasa anual: Préstamos hipotecarios para vivienda",ascending=False).reset_index(drop=True).loc[0:4]
top5_hipo

,Banco,Tasa anual: Préstamos hipotecarios para vivienda
0,Mibanco,15.54
1,BBVA,9.28
2,Crédito,9.27
3,BIF,9.23
4,Promedio,9.13


In [13]:
df3 = df.set_index('Tasa Anual (%)')
df3

,date_info,TRABAJO,BBVA,BANCO DE COMERCIO,BANCO DE CREDITO,BANCO PICHINCHA,BANBIF,STANDARD CHARTERED,BANCO SUDAMERICANO,WIESE SUDAMERIS,...,Banco GNB,Deutsche,Cencosud,CAT,Alfin,B SANTANDER CENTRAL,Azteca *,HSBC(*),Financiero *,BCI
Tasa Anual (%),,,,,,,,,,,,,,,,,,,,,
Avances en Cta.Corriente-,26_2_2004,38.92,-,73.34,-,39.08,-,31.73,-,82.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sobregiros-,26_2_2004,48.79,138.18,89.00,-,68.30,25.00,58.10,-,82.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dsctos. y préstamos hasta 30 días-,26_2_2004,8.19,47.46,4.62,15.87,14.43,-,30.68,39.03,3.19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dsctos. y préstamos 31 - 90 días-,26_2_2004,5.94,21.61,7.19,15.71,10.91,-,11.90,10.58,3.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dsctos. y préstamos 91 - 180 días-,26_2_2004,10.00,19.71,5.34,15.82,10.71,-,9.98,7.56,8.34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Préstamos no Revolventes para libre disponibilidad hasta 360 días,31_8_2022,NaN,22.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,74.42,NaN,NaN,NaN,NaN,-
Préstamos no Revolventes para libre disponibilidad a más de 360 días,31_8_2022,NaN,18.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,69.00,NaN,NaN,NaN,NaN,-
Créditos pignoraticios,31_8_2022,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-,NaN,NaN,NaN,NaN,-


In [14]:
print(df3[300:400].index)

Index(['Medianas Empresas', 'Descuentos', 'Préstamos hasta 30 días',
       'Préstamos de 31 a 90 días', 'Préstamos de 91 a 180 días',
       'Préstamos de 181 a 360 días', 'Préstamos a más de 360 días',
       'Pequeñas Empresas', 'Descuentos', 'Préstamos hasta 30 días',
       'Préstamos de 31 a 90 días', 'Préstamos de 91 a 180 días',
       'Préstamos de 181 a 360 días', 'Préstamos a más de 360 días',
       'Microempresas', 'Tarjetas de Crédito', 'Descuentos',
       'Préstamos Revolventes', 'Préstamos a cuota fija hasta 30 días',
       'Préstamos a cuota fija de 31 a 90 días',
       'Préstamos a cuota fija de 91 a 180 días',
       'Préstamos a cuota fija de 181 a 360 días',
       'Préstamos a cuota fija a más de 360 días', 'Consumo',
       'Tarjetas de Crédito', 'Préstamos Revolventes',
       'Préstamos no Revolventes para automóviles',
       'Préstamos no Revolventes para libre disponibilidad hasta 360 días',
       'Préstamos no Revolventes para libre disponibilidad a más

In [15]:
df3.loc["Consumo"]

,date_info,TRABAJO,BBVA,BANCO DE COMERCIO,BANCO DE CREDITO,BANCO PICHINCHA,BANBIF,STANDARD CHARTERED,BANCO SUDAMERICANO,WIESE SUDAMERIS,...,Banco GNB,Deutsche,Cencosud,CAT,Alfin,B SANTANDER CENTRAL,Azteca *,HSBC(*),Financiero *,BCI
Tasa Anual (%),,,,,,,,,,,,,,,,,,,,,
Consumo,26_2_2021,NaN,35.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Consumo,27_2_2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,31.16,-,83.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Consumo,27_3_2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-,76.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Consumo,27_7_2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Consumo,27_7_2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,81.78,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Consumo,31_8_2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,86.86,NaN,NaN,NaN,NaN,NaN,NaN
Consumo,31_8_2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,78.06,NaN,NaN,NaN,NaN,NaN,NaN
Consumo,31_8_2020,NaN,37.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Part 4.

In [21]:
df.rename(columns = {'Tasa Anual (%)':'tasa_anual_percentage', 
                     'Azteca  *':'Azteca',
                     'HSBC(*)':'HSBC',
                     'Financiero  *':'Financiero'}, inplace = True)

In [32]:
# Quitar espacios y letras mayúsculas de los nombres de las columnas
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.lower()

In [38]:
# Crear bases por cada uno de los bancos
not_requiered_columns = ['tasa_anual_percentage', 'date_info', 'trabajo']
bases = []
for col in df.columns.values:
    if col not in not_requiered_columns:
        data = df[['tasa_anual_percentage', 'date_info', col]]
        bases.append(data)

In [43]:
# Crear el nuevo folder en la direccion especificada
import os

new_path = '../../_output/sbs/group7'
if not os.path.exists( new_path ):
   os.makedirs( new_path )

In [44]:
# Exportar las bases al path indicado
for base in bases:
    cols_names = [col for col in base if col not in not_requiered_columns]
    for col in cols_names:
        new_table_names_path = os.path.join(new_path, f'{col}.xlsx')
        base.to_excel(new_table_names_path)